In [1]:
# High-level CSV EDA
import pandas as pd
from pathlib import Path

base ="NY Crimes"

csv_hist = "NY Crimes/NYPD_Complaint_Data_Historic.csv"
csv_2019 = "NY Crimes/NYPD_Complaint_Data_Historic_2019.csv"

def explore_csv(path: Path, n=5):
    #print(f"\n=== {path.name} ===")
    df = pd.read_csv(path)
    print("Shape:", df.shape)
    print("\nDtypes:\n", df.dtypes)
    print("\nSample rows:\n", df.head(n))
    # Quick NA summary
    na = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
    print("\nTop missing columns (%):\n", na.head(10))
    # Common columns (guarded)
    for c in [ "VIC_AGE_GROUP","VIC_RACE","VIC_SEX","OFNS_DESC","LAW_CAT_CD","BORO_NM" ]:
        if c in df.columns:
            print(f"\nValue counts for {c}:\n", df[c].value_counts(dropna=False).head(10))

explore_csv(csv_hist)
explore_csv(csv_2019)

C:\Users\YODA\AppData\Local\Temp\ipykernel_32480\1321169299.py:12: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Shape: (6983207, 35)

Dtypes:
 CMPLNT_NUM             int64
CMPLNT_FR_DT          object
CMPLNT_FR_TM          object
CMPLNT_TO_DT          object
CMPLNT_TO_TM          object
ADDR_PCT_CD          float64
RPT_DT                object
KY_CD                  int64
OFNS_DESC             object
PD_CD                float64
PD_DESC               object
CRM_ATPT_CPTD_CD      object
LAW_CAT_CD            object
BORO_NM               object
LOC_OF_OCCUR_DESC     object
PREM_TYP_DESC         object
JURIS_DESC            object
JURISDICTION_CODE    float64
PARKS_NM              object
HADEVELOPT            object
HOUSING_PSA           object
X_COORD_CD           float64
Y_COORD_CD           float64
SUSP_AGE_GROUP        object
SUSP_RACE             object
SUSP_SEX              object
TRANSIT_DISTRICT     float64
Latitude             float64
Longitude            float64
Lat_Lon               object
PATROL_BORO           object
STATION_NAME          object
VIC_AGE_GROUP         object
VIC_RACE    

In [2]:
# GeoJSON and GDB layers: structure, CRS, and geometry types
import geopandas as gpd
import fiona
from pathlib import Path

base = Path("NY Crimes") 

layers = [
    base / "NYCHA PSA (Police Service Areas).geojson",
    base / "NYPD Sectors.geojson",
    base / "Police Precincts.geojson",
]

for p in layers:
    print(f"\n=== {p.name} ===")
    gdf = gpd.read_file(p)
    print("Count:", len(gdf), "| CRS:", gdf.crs)
    print("Geometry types:", gdf.geom_type.value_counts().to_dict())
    print("Columns:", list(gdf.columns)[:12])

gdb = base / "ny_data.gdb"
print("\n=== FileGDB layers ===")
print("Layers:", fiona.listlayers(gdb))

# Read known layers from metadata
for layer in ["NYPD_Complaint_Data_Historic_2019_4326", "NYPD_Complaint_Data_Historic_2019"]:
    try:
        gdf = gpd.read_file(gdb, layer=layer)
        print(f"\nLayer: {layer} | Count: {len(gdf)} | CRS: {gdf.crs}")
        print("Geometry types:", gdf.geom_type.value_counts().to_dict() if "geometry" in gdf else "Non-spatial table")
        print("Columns:", list(gdf.columns)[:12])
    except Exception as e:
        print(f"\nLayer: {layer} | Could not read:", e)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E2DA196540>>
Traceback (most recent call last):
  File "C:\Users\YODA\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 796, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\threading.py", line 1521, in enumerate
    with _active_limbo_lock:
SystemError: <built-in method __enter__ of _thread.RLock object at 0x000001E2D637A900> returned a result with an exception set


SystemError: <function Mapping.__contains__ at 0x000001E2D60BA200> returned a result with an exception set